# Heat and Trees

## Urban Heat Islands and Street Trees
In this notebook we'll be exploring the urban heat island effect by looking at the impact on surface temperature of roof color and street trees. We'll be replicating the process described [here](https://urbanspatialanalysis.com/urban-heat-islands-street-trees-in-philadelphia/) but using Python tools rather than ESRI.

**Data sources:** This notebook uses Landsat data from Google Cloud Storage as well as some geographic data from [OpenDataPhilly](https://opendataphilly.org/).

In [ ]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
import geopandas as gpd
import cartopy.crs as ccrs
import hvplot.xarray  # noqa
import hvplot.pandas  # noqa

from geoviews.tile_sources import EsriImagery
from pyproj import CRS

Just some extra info about Landsat data:

In [ ]:
band_info = pd.DataFrame([
        (1,  "Aerosol", " 0.43 - 0.45",    0.440,  "30",         "Coastal aerosol"),
        (2,  "Blue",    " 0.45 - 0.51",    0.480,  "30",         "Blue"),
        (3,  "Green",   " 0.53 - 0.59",    0.560,  "30",         "Green"),
        (4,  "Red",     " 0.64 - 0.67",    0.655,  "30",         "Red"),
        (5,  "NIR",     " 0.85 - 0.88",    0.865,  "30",         "Near Infrared (NIR)"),
        (6,  "SWIR1",   " 1.57 - 1.65",    1.610,  "30",         "Shortwave Infrared (SWIR) 1"),
        (7,  "SWIR2",   " 2.11 - 2.29",    2.200,  "30",         "Shortwave Infrared (SWIR) 2"),
        (8,  "Panc",    " 0.50 - 0.68",    0.590,  "15",         "Panchromatic"),
        (9,  "Cirrus",  " 1.36 - 1.38",    1.370,  "30",         "Cirrus"),
        (10, "TIRS1",   "10.60 - 11.19",   10.895, "100 * (30)", "Thermal Infrared (TIRS) 1"),
        (11, "TIRS2",   "11.50 - 12.51",   12.005, "100 * (30)", "Thermal Infrared (TIRS) 2")],
    columns=['Band', 'Name', 'Wavelength Range (µm)', 'Nominal Wavelength (µm)', 'Resolution (m)', 'Description']).set_index(["Band"])
band_info

## Loading data

For this example, we will be using landsat data stored on Google Cloud Storage. Since these data are accessed via https, there is no guaranteed directory structure, so we will need to specify the url pointing to each file and then iterate over the files to create a concatenated dataset. We use jinja template notation in intake to pass parameters to the `urlpath`.

In [ ]:
cat = intake.open_catalog('catalog.yml')
list(cat)

Let's take a look at what the `google_landsat_band` looks like:

```yml
google_landsat_band:
    description: Landsat bands from Google Cloud Storage
    driver: rasterio
    parameters:
      path:
        description: landsat path
        type: int
      row:
        description: landsat row
        type: int
      product_id:
        description: landsat file id
        type: str
      band:
        description: band
        type: int
    args:
      urlpath: https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/{{ '%03d' % path }}/{{ '%03d' % row }}/{{ product_id }}/{{ product_id }}_B{{ band }}.TIF
      chunks:
        band: 1
        x: 256
        y: 256
```

The following might feel a bit arbitrary, but we have chosen the path and row corresponding to the area over Philadelphia using the [earth explorer](https://earthexplorer.usgs.gov/). We have also found the id of the particular date of interest using the same tool. With these values in hand, we can access parts of each file on Google Cloud Storage. 

In [ ]:
path = 14
row = 32
product_id = 'LC08_L1TP_014032_20160727_20170222_01_T1'

The first step to using intake is to initialize the catalog entry with user parameters to create a `data source`.

In [ ]:
data_source = cat.google_landsat_band(path=path, row=row, product_id=product_id)

From this `data source` we can get a lazily loaded xarray object using dask. To make sure that we can inspect what dask is up to, it can be helpful to create a dask client.

In [ ]:
ds = data_source.to_dask()
ds.name = 'value'

Loading in metadata regarding these particular Landsat images from the associated matlab.txt file.

In [ ]:
def load_google_landsat_metadata(path, row, product_id):
    """Load Landsat metadata for path, row, product_id from Google Cloud Storage
    """
    def parse_type(x):
        try: 
            return eval(x)
        except:
            return x
    
    baseurl = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01'
    suffix = f'{path:03d}/{row:03d}/{product_id}/{product_id}_MTL.txt'
    
    df = intake.open_csv(
        urlpath=f'{baseurl}/{suffix}',
        csv_kwargs={'sep': '=',
                    'header': None,
                    'names': ['index', 'value'],
                    'skiprows': 2,
                    'converters': {'index': (lambda x: x.strip()),
                                   'value': parse_type}}).read()
    metadata = df.set_index('index')['value']
    return metadata

In [ ]:
metadata = load_google_landsat_metadata(path, row, product_id)
metadata.head()

## Sub-setting to area of interest

So far we haven't downloaded any band data. Since we know that we are interested in Philadelphia, we can just take a smaller square of data that covers the extents of the city. First we need to know the projection of the dataset:

In [ ]:
ds.rio.crs.to_proj4()

We'll convert that into something directly usable for later:

In [ ]:
proj_crs= CRS.from_epsg(32618)
proj_crs

In [ ]:
crs = ccrs.UTM(zone=18)

Now if we were just looking for one particular point we could use that point, converted to the coordinate system of the data, and then select the data nearest to it:

In [ ]:
x_center, y_center = crs.transform_point(-75.1652, 39.9526, ccrs.PlateCarree())
nearest_to_center = ds.sel(x=x_center, y=y_center, method='nearest')
print(nearest_to_center.compute())

nearest_to_center.hvplot.line(x='band')

In this case, though, we are interested in a subset of data that covers that city of Philadelphia. So we need some geometry to specify the bounds of the city. We can get a GeoJSON of neighborhood data from [OpenDataPhilly](https://opendataphilly.org/datasets/philadelphia-neighborhoods/).

In [ ]:
geoms = gpd.read_file('data/Neighborhoods_Philadelphia.geojson')

bounds = geoms.geometry.bounds
lower_left_corner_lat_lon = bounds.minx.min(), bounds.miny.min()
upper_right_corner_lat_lon = bounds.maxx.max(), bounds.maxy.max()

print(lower_left_corner_lat_lon, upper_right_corner_lat_lon)

Using the crs defined above, we can transform these lat/lons into map coordinates.

In [ ]:
ll_corner = crs.transform_point(*lower_left_corner_lat_lon, ccrs.PlateCarree())
ur_corner = crs.transform_point(*upper_right_corner_lat_lon, ccrs.PlateCarree())

print(ll_corner, ur_corner)

Then we can use those corners to slice the data. If the subset is empty along x or y, the ordering of the coordinates might not be what you anticipated. Try swapping the order of arguments in the slice.

In [ ]:
subset = ds.sel(x=slice(ll_corner[0], ur_corner[0]), y=slice(ur_corner[1], ll_corner[1]))

We can persist this slice of the dataset in memory for easy use later.

In [ ]:
subset = subset.persist()
subset

To check that we got the right area, we can do a simple plot of one of the bands and overlay the neighborhoods on top of it. We'll use [hvplot](https://hvplot.pyviz.org) to quickly create a [holoviews](https://holoviews.org) object rendered in [bokeh](https://bokeh.org).

In [ ]:
geoms.head()

In [ ]:
band_plot = subset.mean('band').hvplot(x='x', y='y', rasterize=True, crs=crs, cmap='gray')
hood_plot = geoms.hvplot(geo=True, alpha=.5, c='mapname', legend=False, frame_height=450)

band_plot * hood_plot

## Calculate NDVI

We'll calculate NDVI but we won't yet do any computations -- our bands are actually dask arrays, which allow for lazy computation.

In [ ]:
subset = subset.where(subset > 0)
NDVI = (subset.sel(band=5) - subset.sel(band=4)) / (subset.sel(band=5) + subset.sel(band=4))
NDVI = NDVI.where(NDVI < np.inf)
NDVI

In order to visualize NDVI, the data will need to be loaded and the NDVI computed.

In [ ]:
NDVI.hvplot(x='x', y='y', crs=crs, rasterize=True, cmap='viridis', frame_height=450)

## Calculate land surface temperature

Given the NDVI calculated above, we can determine land surface temperature. For ease, we'll use some *top of atmosphere* calculations that have already been written up as Python functions as part of rasterio work in the [rio_toa](https://github.com/mapbox/rio-toa/tree/master) module. We will reproduce the `brightness_temp` and `toa_utils` functions from the `rio_toa` module here to make it easier and faster.

In [ ]:
def brightness_temp(img, ML, AL, K1, K2, src_nodata=0):
    """Calculate brightness temperature of Landsat 8 combining the calculations of radiance and temperature.

    Parameters:
        img (ndarray): Input pixels array.
        ML (float): Multiplicative rescaling factor from scene metadata.
        AL (float): Additive rescaling factor from scene metadata.
        K1 (float): Thermal conversion constant from scene metadata.
        K2 (float): Thermal conversion constant from scene metadata.
        src_nodata (int, optional): Value representing 'no data' in input array.

    Returns:
        ndarray: Array of at-satellite brightness temperature in degrees Kelvin.
    """
    L = ML * img.astype(np.float32) + AL
    L[img == src_nodata] = np.NaN

    T = K2 / np.log((K1 / L) + 1)

    return T

In [ ]:
def temp_rescale(arr, temp_scale):
    """Converts the `arr` in Kelvin (K) to the specified `temp_scale`- Kelvin(K), Farenheit(F) or Celcius(C)."""
    if temp_scale == 'K':
        return arr

    elif temp_scale == 'F':
        return arr * (9 / 5.0) - 459.67

    elif temp_scale == 'C':
        return arr - 273.15

    else:
        raise ValueError('%s is not a valid temperature scale'
                         % (temp_scale))

We'll also need to specify one more for transforming satellite temperature (brightness temp) to land surface temperature. For these calculations we'll use both Thermal Infrared bands - 10 and 11.

In [ ]:
def land_surface_temp(BT, w, NDVI):
    """Calculate land surface temperature of Landsat 8
    
    temp = BT/1 + w * (BT /p) * ln(e)
    
    BT = At Satellite temperature (brightness)
    w = wavelength of emitted radiance (μm)
    
    where p = h * c / s (1.439e-2 mK)
    
    h = Planck's constant (Js)
    s = Boltzmann constant (J/K)
    c = velocity of light (m/s)
    """
    h = 6.626e-34
    s = 1.38e-23
    c = 2.998e8
    
    p = (h * c / s) * 1e6
    
    Pv = (NDVI - NDVI.min() / NDVI.max() - NDVI.min())**2
    e = 0.004 * Pv + 0.986
    
    return BT / 1 + w * (BT / p) * np.log(e)

Now we'll set up a helper function to retrieve all the parameters from the metadata and general Landsat info table, and calculate the land surface temperature for bands 10 and 11.

In [ ]:
def land_surface_temp_for_band(band, data, units='F'):
    # params from general Landsat info
    w = band_info.loc[band]['Nominal Wavelength (µm)']
    
    # params from specific Landsat data text file for these images
    ML = metadata[f'RADIANCE_MULT_BAND_{band}']
    AL = metadata[f'RADIANCE_ADD_BAND_{band}']
    K1 = metadata[f'K1_CONSTANT_BAND_{band}']
    K2 = metadata[f'K2_CONSTANT_BAND_{band}']
    
    at_satellite_temp = brightness_temp(data.sel(band=band).values, ML, AL, K1, K2)
    
    temp = land_surface_temp(at_satellite_temp, w, NDVI).compute()
    return temp_rescale(temp, units)

In [ ]:
temp_10_f = land_surface_temp_for_band(10, subset)
temp_11_f = land_surface_temp_for_band(11, subset)

temp_f = xr.concat([temp_10_f, temp_11_f], 
                   dim=xr.DataArray([10,11], name='band', dims=['band']))
temp_f

Compare the results from the two different bands, noticing that the colorbars are different.

In [ ]:
temp_f.hvplot(x='x', y='y', groupby='band', cmap='fire_r', 
              crs=crs, rasterize=True, project=True, frame_height=350).layout()

We'll take the mean of the calculated land surface temperature for each of the bands and mimic the colormap used in the project that we are duplicating.

In [ ]:
mean_temp_f = temp_f.mean(dim='band')
mean_temp_f.hvplot(x='x', y='y', title='Mean Surface Temp (F)', crs=crs, tiles='EsriImagery',
                   frame_height=450, project=True, cmap='rainbow', alpha=.5, legend=False)

Notice how the hot spots are located over warehouse roofs and parking lots. This becomes even more visible when just the temperatures greater than 90F are displayed. To show this, we'll make a special colormap that just includes high intensity reds that are found at the top of the `fire_r` colormap.

In [ ]:
import colorcet as cc

special_cmap = cc.fire[::-1][90:]

In [ ]:
thresholded_temp_p = (mean_temp_f.where(mean_temp_f > 90)
    .hvplot(x='x', y='y', title='Mean Temp (F) > 90',
            cmap=special_cmap, crs=crs, frame_width=400,
            frame_height=450, legend=False)
    .redim(value='Temperature (F)'))

thresholded_temp_p + thresholded_temp_p.opts(alpha=.3, data_aspect=None) * EsriImagery

## Adding in the Street Tree data

OpenDataPhilly released an inventory of all the street trees in the city. Street trees are trees that are planted along streets, not those in parks and private property. The original analysis considered these 100,000 points too many to plot, but that's nothing to `datashader`, which is happy with billions of points even on a laptop.

It is hypothesized that where there are more trees the land surface temperature will be less extreme. To explore this, we will overlay street trees with the thresholded land surface temperature:

In [ ]:
trees_gdf = gpd.read_file('data/PPR_Tree_Inventory_2022.geojson')
trees_gdf.head()

The trees plot can be generated straight from the geopandas dataframe, but that is rather slow. By inspecting the dataframe, we can see that each tree is represented as a point. We can create a simpler pandas dataframe with `lat` as one column and `lon` as the other.

In [ ]:
trees_df = pd.DataFrame({'Longitude': trees_gdf.geometry.x, 'Latitude': trees_gdf.geometry.y})
trees_df.head()

Here, we use the `dynspread` parameter that helps us to dynamically calculate the points to display based on the local density of the points. This is more evident when you zoom into a sparse region of the plot. We also set the `threshold` parameter to control the fraction of the maximum density at which spreading starts, as well as `max_px` which sets the maximum number of pixels by which points can be spread.

See [datashader.spreading](https://datashader.org/getting_started/Pipeline.html#spreading) and [dynspread](https://datashader.org/api.html#datashader.transfer_functions.dynspread) for more details:

In [ ]:
tree_p = trees_df.hvplot.points('Longitude', 'Latitude', title='Street Tree Density',
                                geo=True, rasterize=True, dynspread=True, max_px=5,
                                 threshold=0.9, frame_height=450, cmap='Greens', cnorm='eq_hist')

thresholded_temp_p.opts(alpha=.5) * tree_p.opts(alpha=.5)